# Problema de Composição de PCs

## Formulação Matemática

### Função Objetivo
$$\max \sum_{i \in I} \sum_{pc \in PC} b_i \cdot x_{i,pc} \quad (1)$$

### Restrições Principais

**Restrição de Capacidade do Galpão:**
$$\sum_{i \in I} \sum_{pc \in PC} w_i \cdot x_{i,pc} \leq C \quad (2)$$

**Restrição de Estoque:**
$$\sum_{pc \in PC} x_{i,pc} \leq s_i \quad \forall i \in I \quad (3)$$

**Atendimento de Demanda Mínima:**
$$p_{pc} \geq d_{pc} \quad \forall pc \in PC \quad (4)$$

**Compatibilidade de Componentes:**
$$x_{i,pc} = 0 \quad \forall i \notin Comp_{pc}, \forall pc \in PC \quad (5)$$

**Seleção Única por Categoria:**
$$\sum_{i \in Cat_k \cap Comp_{pc}} y_{i,pc} = 1 \quad \forall pc \in PC, \forall k \in K \quad (6)$$

### Restrições de Linearização

**Linearização do Produto (Big-M):**
$$z_{i,pc} \leq M \cdot y_{i,pc} \quad \forall i \in I, \forall pc \in PC \quad (7)$$

$$z_{i,pc} \leq p_{pc} \quad \forall i \in I, \forall pc \in PC \quad (8)$$

$$z_{i,pc} \geq p_{pc} - M \cdot (1 - y_{i,pc}) \quad \forall i \in I, \forall pc \in PC \quad (9)$$

**Quantidade Exata:**
$$x_{i,pc} = q_k \cdot z_{i,pc} \quad \forall i \in Cat_k, \forall pc \in PC, \forall k \in K \quad (10)$$

### Domínio das Variáveis
$$x_{i,pc} \in \mathbb{Z}_+ \quad \forall i \in I, \forall pc \in PC \quad (11)$$
$$p_{pc} \in \mathbb{Z}_+ \quad \forall pc \in PC \quad (12)$$
$$y_{i,pc} \in \{0, 1\} \quad \forall i \in I, \forall pc \in PC \quad (13)$$
$$z_{i,pc} \in \mathbb{Z}_+ \quad \forall i \in I, \forall pc \in PC \quad (14)$$

## Parâmetros do Modelo

- **$I$**: Conjunto de itens/componentes disponíveis
- **$PC$**: Conjunto de configurações de PCs
- **$K$**: Conjunto de categorias essenciais (Gabinete, Placa Mãe, Processador, Memória RAM, SSD)
- **$b_i$**: Valor/benefício do item $i$ (Media_Vendas)
- **$w_i$**: Peso/espaço ocupado pelo item $i$ (Espaco_Galpao para Gabinetes)
- **$s_i$**: Estoque disponível do item $i$
- **$d_{pc}$**: Demanda mínima da configuração $pc$
- **$q_k$**: Quantidade básica necessária da categoria $k$ (sempre 1)
- **$C$**: Capacidade máxima do galpão (14.185.344,5 m³)
- **$M$**: Constante Big-M para linearização (10.000)
- **$Comp_{pc}$**: Conjunto de componentes compatíveis com a configuração $pc$
- **$Cat_k$**: Conjunto de componentes da categoria $k$

## Variáveis de Decisão

- **$x_{i,pc}$**: Quantidade do componente $i$ usado na configuração $pc$
- **$p_{pc}$**: Número de PCs produzidos da configuração $pc$
- **$y_{i,pc}$**: Variável binária de seleção do componente $i$ para configuração $pc$
- **$z_{i,pc}$**: Variável auxiliar para linearização ($p_{pc} \times y_{i,pc}$)

In [2]:
import pandas as pd
import pyomo.environ as pyo

# Configurações
DATA_PATH = 'dados_composicao.xlsx'
RESULTS_PATH = 'resultados_otimizados.xlsx'
Path_Solver = 'C:/cbc/bin/cbc'
CAPACIDADE_GALPAO = 3111430.40


DEMANDAS_PCS = {
    'Comp_01_Valianty': 20, 'Comp_02_Valianty': 25, 'Comp_03_Valianty': 50,
    'Comp_01_Yon': 15,
    'Comp_04_Yon': 10, 'Comp_05_Yon': 5
}

COMPONENTES_ESSENCIAIS = {
    'Placa Mae': 1, 'Processador': 1, 'Memoria Ram': 1,
    'SSD': 1, 'Gabinete': 1
}

# Carregar dados
print("Carregando dados de configuração...")
df = pd.read_excel(DATA_PATH)
df = df[df['Estoque'] > 0].copy()

pecas_validas = df.set_index('ID').to_dict(orient='index')
for k, v in pecas_validas.items():
    v['ID'] = k


# VERIFICAÇÃO DE VIABILIDADE

compatibilidade_total = {
    pc: df[df[pc] == 1]['ID'].tolist()
    for pc in DEMANDAS_PCS if pc in df.columns
}

configs_viaveis = {}
for pc, componentes in compatibilidade_total.items():
    categorias_disponiveis = df[df['ID'].isin(componentes)]['Categoria'].unique()
    if all(cat in categorias_disponiveis for cat in COMPONENTES_ESSENCIAIS):
        configs_viaveis[pc] = DEMANDAS_PCS[pc]

print(f"Configurações analisadas: {len(configs_viaveis)} viáveis de {len(DEMANDAS_PCS)} totais")


# PREPARAÇÃO DOS PARÂMETROS

valor_item = {k: v['Media_Vendas'] for k, v in pecas_validas.items()}
estoque_item = {k: v['Estoque'] for k, v in pecas_validas.items()}

categoria_item = {k: v['Categoria'] for k, v in pecas_validas.items()}

peso_item = {k: v['Espaco_Galpao'] if v['Categoria'] == 'Gabinete' else 0 for k, v in pecas_validas.items()}
compatibilidade_viaveis = {pc: comp_list for pc, comp_list in compatibilidade_total.items() if pc in configs_viaveis}

# Big M para linearização
BIG_M = 10000


# MODELO

model = pyo.ConcreteModel("Otimizacao_PC_Linear")

# Conjuntos
model.ITENS = pyo.Set(initialize=list(pecas_validas.keys()))
model.CATEGORIAS = pyo.Set(initialize=list(COMPONENTES_ESSENCIAIS.keys()))
model.CONFIGS_PC = pyo.Set(initialize=list(configs_viaveis.keys()))

# Parâmetros
model.valor = pyo.Param(model.ITENS, initialize=valor_item)
model.peso = pyo.Param(model.ITENS, initialize=peso_item)
model.estoque_max = pyo.Param(model.ITENS, initialize=estoque_item)
model.demanda_pc = pyo.Param(model.CONFIGS_PC, initialize=configs_viaveis)
model.qtd_basica = pyo.Param(model.CATEGORIAS, initialize=COMPONENTES_ESSENCIAIS)

# Variáveis de Decisão
model.x_config = pyo.Var(model.ITENS, model.CONFIGS_PC, domain=pyo.NonNegativeIntegers)
model.pcs_produzidos = pyo.Var(model.CONFIGS_PC, domain=pyo.NonNegativeIntegers)
model.y_selecao = pyo.Var(model.ITENS, model.CONFIGS_PC, domain=pyo.Binary)

# linearizar produto pcs_produzidos * y_selecao
model.z_produto = pyo.Var(model.ITENS, model.CONFIGS_PC, domain=pyo.NonNegativeIntegers)

# Função Objetivo
def objetivo_rule(m):
    return sum(m.valor[i] * sum(m.x_config[i, pc] for pc in m.CONFIGS_PC) for i in m.ITENS)
model.objetivo = pyo.Objective(rule=objetivo_rule, sense=pyo.maximize)

# Restrições Básicas
def capacidade_rule(m):
    return sum(m.peso[i] * sum(m.x_config[i, pc] for pc in m.CONFIGS_PC) for i in m.ITENS) <= CAPACIDADE_GALPAO
model.capacidade_galpao = pyo.Constraint(rule=capacidade_rule)

def estoque_rule(m, i):
    return sum(m.x_config[i, pc] for pc in m.CONFIGS_PC) <= m.estoque_max[i]
model.limite_estoque = pyo.Constraint(model.ITENS, rule=estoque_rule)

def demanda_rule(m, pc):
    return m.pcs_produzidos[pc] >= m.demanda_pc[pc]
model.atender_demanda = pyo.Constraint(model.CONFIGS_PC, rule=demanda_rule)

def compatibilidade_rule(m, i, pc):
    if i not in compatibilidade_viaveis[pc]:
        return m.x_config[i, pc] == 0
    return pyo.Constraint.Skip
model.compatibilidade_componentes = pyo.Constraint(model.ITENS, model.CONFIGS_PC, rule=compatibilidade_rule)


# RESTRIÇÕES

# RESTRIÇÃO 1: Exatamente um componente por categoria por configuração
def um_componente_por_categoria_rule(m, pc, cat):
    componentes_compativeis = [i for i in compatibilidade_viaveis[pc] if categoria_item.get(i) == cat]
    if not componentes_compativeis:
        return m.pcs_produzidos[pc] == 0
    return sum(m.y_selecao[i, pc] for i in componentes_compativeis) == 1
model.um_componente_por_categoria = pyo.Constraint(model.CONFIGS_PC, model.CATEGORIAS, rule=um_componente_por_categoria_rule)

# RESTRIÇÕES PARA LINEARIZAÇÃO DO PRODUTO (pcs_produzidos * y_selecao)
# z_produto[i,pc] representa pcs_produzidos[pc] * y_selecao[i,pc]

# Restrição 1: z_produto <= BIG_M * y_selecao (se não selecionado, z = 0)
def linearizacao_1_rule(m, i, pc):
    if i in compatibilidade_viaveis[pc]:
        return m.z_produto[i, pc] <= BIG_M * m.y_selecao[i, pc]
    else:
        return m.z_produto[i, pc] == 0
model.linearizacao_1 = pyo.Constraint(model.ITENS, model.CONFIGS_PC, rule=linearizacao_1_rule)

# Restrição 2: z_produto <= pcs_produzidos (z não pode exceder pcs_produzidos)
def linearizacao_2_rule(m, i, pc):
    if i in compatibilidade_viaveis[pc]:
        return m.z_produto[i, pc] <= m.pcs_produzidos[pc]
    return pyo.Constraint.Skip
model.linearizacao_2 = pyo.Constraint(model.ITENS, model.CONFIGS_PC, rule=linearizacao_2_rule)

# Restrição 3: z_produto >= pcs_produzidos - BIG_M * (1 - y_selecao) (se selecionado, z = pcs_produzidos)
def linearizacao_3_rule(m, i, pc):
    if i in compatibilidade_viaveis[pc]:
        return m.z_produto[i, pc] >= m.pcs_produzidos[pc] - BIG_M * (1 - m.y_selecao[i, pc])
    return pyo.Constraint.Skip
model.linearizacao_3 = pyo.Constraint(model.ITENS, model.CONFIGS_PC, rule=linearizacao_3_rule)

# RESTRIÇÃO PRINCIPAL: Quantidade exata
def quantidade_exata_linearizada_rule(m, i, pc):
    if i in compatibilidade_viaveis[pc]:
        categoria = categoria_item.get(i)
        if categoria in COMPONENTES_ESSENCIAIS:
            # LINEARIZADO: x_config = qtd_basica * z_produto (em vez de qtd_basica * pcs_produzidos * y_selecao)
            return m.x_config[i, pc] == m.qtd_basica[categoria] * m.z_produto[i, pc]
    return pyo.Constraint.Skip
model.quantidade_exata_linearizada = pyo.Constraint(model.ITENS, model.CONFIGS_PC, rule=quantidade_exata_linearizada_rule)

# RESOLUÇÃO ---

solver = pyo.SolverFactory('cbc', executable = Path_Solver)
solver.options['seconds'] = 300
result = solver.solve(model, tee=False)

if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Otimização concluída com sucesso\n")

    # Cálculos principais
    espaco_utilizado = sum(pyo.value(model.peso[i] * sum(model.x_config[i, pc] for pc in model.CONFIGS_PC)) for i in model.ITENS)
    taxa_utilizacao = espaco_utilizado / CAPACIDADE_GALPAO
    total_pcs_produzidos = sum(pyo.value(model.pcs_produzidos[pc]) for pc in model.CONFIGS_PC)

    print("=" * 80)
    print("PLANO DE PRODUÇÃO")
    print("=" * 80)

    print(f"\nUTILIZAÇÃO DO GALPÃO:")
    print(f"  Espaço utilizado: {espaco_utilizado:,.2f} m³")
    print(f"  Capacidade total: {CAPACIDADE_GALPAO:,.2f} m³")
    print(f"  Taxa de ocupação: {taxa_utilizacao:.1%}")

    print(f"\nRESUMO GERAL:")
    print(f"  Total de PCs programados: {int(total_pcs_produzidos)} unidades")
    print(f"  Configurações ativas: {len([pc for pc in model.CONFIGS_PC if pyo.value(model.pcs_produzidos[pc]) > 0])}")

    print(f"\nCONFIGURAÇÕES DE PRODUÇÃO:")
    print("-" * 80)

    resultados_detalhados = []
    violacoes_composicao = 0

    for pc in model.CONFIGS_PC:
        pcs_produzidos = int(pyo.value(model.pcs_produzidos[pc]))
        demanda_original = configs_viaveis[pc]

        if pcs_produzidos > 0:
            nome_config = pc.replace('Comp_', '').replace('_', ' ')

            print(f"\n{nome_config.upper()}")
            print(f"  Quantidade: {pcs_produzidos} unidades")
            print(f"  Demanda mínima: {demanda_original} unidades")
            print(f"  Status: {'Atendida' if pcs_produzidos >= demanda_original else 'Pendente'}")
            print(f"  Configuração padrão única:")

            # Verificar cada categoria
            for categoria in ['Gabinete', 'Placa Mae', 'Processador', 'Memoria Ram', 'SSD']:
                componentes_categoria = [i for i in compatibilidade_viaveis[pc] if categoria_item.get(i) == categoria]

                # Encontrar componente selecionado
                componente_selecionado = None
                qtd_total_categoria = 0

                for i in componentes_categoria:
                    qtd_usado = pyo.value(model.x_config[i, pc])
                    if qtd_usado > 0:
                        qtd_total_categoria += qtd_usado
                        if pyo.value(model.y_selecao[i, pc]) > 0.5:
                            componente_selecionado = i

                # Verificar composição
                qtd_esperada = COMPONENTES_ESSENCIAIS[categoria] * pcs_produzidos
                composicao_correta = (qtd_total_categoria == qtd_esperada)

                if not composicao_correta:
                    violacoes_composicao += 1

                if componente_selecionado:
                    comp_info = pecas_validas[componente_selecionado]
                    descricao_limpa = comp_info['Descrição'].replace('(Oem', '').replace('Oem', '').strip()

                    status_comp = "✓" if composicao_correta else f"✗ ({int(qtd_total_categoria)}/{int(qtd_esperada)})"

                    print(f"    {categoria}: {descricao_limpa} {status_comp}")
                    print(f"      ID: {componente_selecionado} | Quantidade: {int(qtd_total_categoria)} unidades")

                    resultados_detalhados.append({
                        'Configuracao': nome_config,
                        'Categoria': categoria,
                        'ID_Componente': componente_selecionado,
                        'Descricao_Componente': descricao_limpa,
                        'Quantidade_Por_PC': COMPONENTES_ESSENCIAIS[categoria],
                        'Quantidade_Total': int(qtd_total_categoria),
                        'PCs_Produzidos': pcs_produzidos,
                        'Composicao_Correta': composicao_correta,
                        'Demanda_Minima': demanda_original
                    })
else:
    print("O solver não encontrou uma solução ótima.")
    print(f"Status do Solver: {result.solver.status}")
    print(f"Condição de Terminação: {result.solver.termination_condition}")

Carregando dados de configuração...
Configurações analisadas: 6 viáveis de 6 totais
Otimização concluída com sucesso

PLANO DE PRODUÇÃO

UTILIZAÇÃO DO GALPÃO:
  Espaço utilizado: 3,093,552.00 m³
  Capacidade total: 3,111,430.40 m³
  Taxa de ocupação: 99.4%

RESUMO GERAL:
  Total de PCs programados: 155 unidades
  Configurações ativas: 6

CONFIGURAÇÕES DE PRODUÇÃO:
--------------------------------------------------------------------------------

01 VALIANTY
  Quantidade: 47 unidades
  Demanda mínima: 20 unidades
  Status: Atendida
  Configuração padrão única:
    Gabinete: GABINETE VALIANTY 255-10 FTE 230W ✓
      ID: 18187 | Quantidade: 47 unidades
    Placa Mae: Placa Mae Yon 1155 H61G578 Hdmi Glan V2 ✓
      ID: 19247 | Quantidade: 47 unidades
    Processador: Processador Intel 1155 I5-2400 Tray ✓
      ID: 14668 | Quantidade: 47 unidades
    Memoria Ram: Memoria So-Dimm Ddr3 08Gb/1600 Hiker ✓
      ID: 20215 | Quantidade: 47 unidades
    SSD: SSD 0240GB SATA PATRIOT BURST ELITE ✓
  